We have a problem

We have a problem.

The training set is small and if we split it into a partial set and a validation set it will be even smaller

And the validation set would be tiny

The validation scores - on a tiny validation set - would be very sensitive to the constitution of that set

The training set is small and if we split it into a partial set and a validation set it will be even smaller. And the validation set would be tiny. The validation scores - on a tiny validation set - would be very sensitive to the constitution of that set

**$K$-fold cross-validation** is a common solution

The data is spilt into $K$ partitions (usually 4 or 5)

$K$ identical models are instantiated

**$K$-fold cross-validation** is a common solution. The data is spilt into $K$ partitions (usually 4 or 5) and 
$K$ identical models are instantiated.

Each network is trained and validated on a *fold*

 Fold | Training   | Validation 
------|:------------|----------
 1    | 2, 3... $K$ |1
 2    | 1, 3... $K$ |2
 ...  | ...         | 
 $K$  | 1, 2... $K-1$  |$K$
 
The validation score is the average of the $K$ validation scores

This technique mitigates validation variance

Each network is trained on a fold. A network is trained on partitions 2, 3, 4 up to K and validated on partition 1. This is fold one. A fresh network is trained on partitions 1, 3, 4 up to K and validated om partition 2. This is fold 2. Fold i training means a network is trained on all training data except partition i, and validated on partition i. The idea is to reduce the sensitivity of validation to a small set. Validation might yield quite different results depending on the particular choice of validation set.

In [1]:
from tensorflow.keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

mean = train_data.mean(axis = 0)
train_data -= mean # shift
std = train_data.std(axis = 0)
train_data /= std # rescale

test_data -= mean
test_data /= std

from tensorflow.keras import models
from tensorflow.keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, 
                           activation = 'relu', 
                           input_shape = (train_data.shape[1], )))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mae'])
    
    return model

In [2]:
import numpy as np

K = 4
num_val_samples = len(train_data) // K
num_epochs = 100
all_scores = []

for i in range(K):
    print('processing fold', i)
    
    # Prepare the validation data: data from partition i
    a, b = i * num_val_samples, (i + 1) * num_val_samples
    val_data = train_data[a : b]
    val_targets = train_targets[a : b]
    
    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate([train_data[:a], train_data[b:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:a], train_targets[b:]], axis=0)
    
    # Build the Keras model (already compiled)
    model = build_model()
    
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
    
    # Evaluate the model on the validation datam
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold 0
processing fold 1
processing fold 2
processing fold 3


The K-fold training code. K is 4. Training and validation occurs in a loop: a network is instantiated at each pass and the validation and partial training set is sliced from the training set. The validation mean absolute error is added to the `all_scores` list.

We are looking at the validation score at termination, not at each epoch (no `history` object)

Long training run - 100 epochs

We are looking at the validation score at termination, not at each epoch so there is no `history` object. The  training run, at 100 epochs, is long.

In [3]:
print(all_scores)
print(np.mean(all_scores))

[2.3290622234344482, 2.3244190216064453, 2.7118148803710938, 2.718132972717285]
2.520857274532318


The fold MAE scores, and their mean.

In [4]:
print(min(all_scores), ' to ', max(all_scores))

2.3244190216064453  to  2.718132972717285


The MAE ranges from...to...justifying our cautious approach.

In [5]:
print('$' + str(int(round(np.mean(all_scores) * 1000))))

$2521


The average MAE is significant since the prices range from \\$10,000 to \\$50,000